In [1]:
%load_ext autoreload
%autoreload 2
import torch
import jax

# Torch Model

In [2]:
# %cd ~/InternVL/classification/
# from models.build import build_model
# # initialize config
# import ml_collections
# config_dict = {
#     "MODEL": {
#         "TYPE": "intern_vit_6b",
#         "NUM_CLASSES": 0,
#         "DROP_PATH_RATE": 0.0,
#         "INTERN_VIT_6B": {
#             "PATCH_SIZE": 16,
#             "EMBED_DIM": 768,
#             "NUM_HEADS": 12,
#             "MLP_RATIO": 4,
#             "QKV_BIAS": True,
#             "INIT_VALUES": 1e-6,
#             "QK_NORMALIZATION": False,
#             "DEPTH": 12,
#             "USE_FLASH_ATTN": False,
#             "PRETRAIN_SIZE": 224,
#             "PRETRAINED": None,
#             "CLS_TARGET": "clip_projector",
#             "HEAD_NORM_TYPE": "none",
#             "FREEZE_VIT": False,
#         },
#     },
#     "DATA": {
#         "IMG_SIZE": 224
#     },
#     "TRAIN": {
#         "USE_CHECKPOINT": False
#     }
# }
# config = ml_collections.ConfigDict(config_dict)
# model = build_model(config=config)
%cd ~/unilm/beit/
from modeling_pretrain import beit_base_patch16_224_8k_vocab
model = beit_base_patch16_224_8k_vocab(pretrained=False)
# Input Initialization
dummy_img = torch.randn(1, 3, 224, 224)
bool_masked_pos = torch.randn(1,196).bool()
out = model.forward_features(dummy_img,bool_masked_pos)
out.shape, model

[Errno 2] No such file or directory: '/home/austinwang/unilm/beit/'
/home/austinwang/austin_big_vision/scripts


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


ModuleNotFoundError: No module named 'modeling_pretrain'

# Trucated normalization comparison

In [2]:
# Flax
import flax
import jax.numpy as jnp
class flax_foo(flax.linen.Module):
    feature_dim: int
    kernel_init: flax.linen.initializers.Initializer = flax.linen.initializers.truncated_normal(1.0, dtype=jnp.float32, lower=-0.02, upper=0.02)
    bias_init: flax.linen.initializers.Initializer = flax.linen.initializers.zeros
    def setup(self):
        self.dense = flax.linen.Dense(features=self.feature_dim, kernel_init=self.kernel_init, bias_init=self.bias_init)
    def __call__(self, x):
        return self.dense(x)

# torch 
from timm.models.layers import trunc_normal_
def trunc_normal(param): return trunc_normal_(param, mean=0.0, std=0.02, a=-0.02, b=0.02)
class torch_foo(torch.nn.Module):
    def __init__(self, feature_dim: int):
        super().__init__()
        self.dense = torch.nn.Linear(feature_dim, feature_dim)
        trunc_normal(self.dense.weight)
        torch.nn.init.zeros_(self.dense.bias)
    def forward(self, x):
        return self.dense(x)


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Input Initialization
feature_dim = 2048
torch.manual_seed(0)
x = torch.ones(1, feature_dim, requires_grad=False)
x_flax = jnp.array(x)

# model initialization
flax_model = flax_foo(feature_dim)
params = flax_model.init(jax.random.PRNGKey(0), x_flax)
torch_model = torch_foo(feature_dim)
torch_params = torch_model.state_dict()


In [4]:
# params['params']['dense']['kernel'], torch_params['dense.weight']

In [5]:
params['params']['dense']['kernel'].std().item(), torch_params['dense.weight'].std().item()

(0.01154741458594799, 0.010793177410960197)

In [6]:
# forward pass
flax_out = flax_model.apply(params, x_flax)
torch_out = torch_model(x)